In [7]:
import requests
from bs4 import BeautifulSoup
import time
import json

BASE_URL = "https://reliefweb.int"

def get_disaster_links(page_num):
    """
    Fetch a list page and extract all disaster article URLs.
    """
    url = f"{BASE_URL}/disasters?page={page_num}"
    print(f"Fetching list page: {url}")
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching page {page_num}: {response.status_code}")
        return []
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.select("article.rw-river-article")
    links = []
    for article in articles:
        a_tag = article.select_one("h3.rw-river-article__title a")
        if a_tag:
            link = a_tag.get("href")
            if link and not link.startswith("http"):
                link = BASE_URL + link
            links.append(link)
    return links

def get_all_links():
    """
    Loop through list pages until no new articles are found.
    """
    all_links = []
    page_num = 0
    while True:
        links = get_disaster_links(page_num)
        if not links:
            print(f"No links found on page {page_num}. Ending pagination.")
            break
        print(f"Found {len(links)} links on page {page_num}")
        all_links.extend(links)
        page_num += 1
        if page_num >= 182:
            break
        # time.sleep(1)  # Respectful delay
    return list(set(all_links))  # remove duplicates if any

def scrape_disaster_detail(url):
    """
    For a given disaster article URL, scrape detail page and extract:
      - Title
      - Status (e.g., "Past Disaster", "Ongoing")
      - Disaster description (from section with id "overview")
      - Affected countries (from section with id "countries")
      - Related disasters (from section with id "disasters", if present)
    """
    print(f"Scraping detail page: {url}")
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching detail page: {url}")
        return None
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Title: typically in an h1 with class "rw-page-title"
    title_el = soup.select_one("h1.rw-page-title")
    title = title_el.get_text(strip=True) if title_el else "No title"
    
    # Status: from meta data (dd element with a class containing '--status')
    status_el = soup.select_one("dd[class*='rw-entity-meta__tag-value--status']")
    status = status_el.get_text(strip=True) if status_el else "No status"
    
    # Disaster description: from section id "overview" and then div with id "overview-content"
    desc_el = soup.select_one("section#overview div#overview-content")
    description = desc_el.get_text(separator=" ", strip=True) if desc_el else "No description"
    
    # Affected countries: from section with id "countries"
    affected_countries = []
    country_articles = soup.select("section#countries article.rw-river-article--country")
    for ca in country_articles:
        country_link = ca.select_one("h3.rw-river-article__title a")
        if country_link:
            affected_countries.append(country_link.get_text(strip=True))
    if not affected_countries:
        affected_countries = ["Not specified"]
    
    # Related disasters: from section with id "disasters"
    related_disasters = []
    related_articles = soup.select("section#disasters article.rw-river-article--disaster")
    for ra in related_articles:
        rel_a = ra.select_one("h3.rw-river-article__title a")
        if rel_a:
            rel_title = rel_a.get_text(strip=True)
            rel_link = rel_a.get("href")
            if rel_link and not rel_link.startswith("http"):
                rel_link = BASE_URL + rel_link
            related_disasters.append({
                "title": rel_title,
                "link": rel_link
            })
    
    return {
        "url": url,
        "title": title,
        "status": status,
        "description": description,
        "affected_countries": affected_countries,
        "related_disasters": related_disasters
    }

# def main():
#     # Phase 1: Collect all article URLs from the list pages.
#     article_links = get_all_links()
#     print(f"Total unique disaster articles found: {len(article_links)}")
    
#     # Phase 2: Visit each article and extract detailed information.
#     disaster_details = []
#     for link in article_links:
#         detail = scrape_disaster_detail(link)
#         if detail:
#             disaster_details.append(detail)
#         time.sleep(1)  # Polite delay between requests
    
#     # Save collected data to JSON.
#     with open("disaster_details.json", "w", encoding="utf-8") as f:
#         json.dump(disaster_details, f, ensure_ascii=False, indent=2)
#     print("Scraping complete. Data saved to disaster_details.json.")

# if __name__ == "__main__":
#     main()


In [8]:
# Phase 1: Collect all article URLs from the list pages.
article_links = get_all_links()
print(f"Total unique disaster articles found: {len(article_links)}")

Fetching list page: https://reliefweb.int/disasters?page=0
Found 59 links on page 0
Fetching list page: https://reliefweb.int/disasters?page=1
Found 59 links on page 1
Fetching list page: https://reliefweb.int/disasters?page=2
Found 59 links on page 2
Fetching list page: https://reliefweb.int/disasters?page=3
Found 59 links on page 3
Fetching list page: https://reliefweb.int/disasters?page=4
Found 59 links on page 4
Fetching list page: https://reliefweb.int/disasters?page=5
Found 59 links on page 5
Fetching list page: https://reliefweb.int/disasters?page=6
Found 59 links on page 6
Fetching list page: https://reliefweb.int/disasters?page=7
Found 59 links on page 7
Fetching list page: https://reliefweb.int/disasters?page=8
Found 59 links on page 8
Fetching list page: https://reliefweb.int/disasters?page=9
Found 59 links on page 9
Fetching list page: https://reliefweb.int/disasters?page=10
Found 59 links on page 10
Fetching list page: https://reliefweb.int/disasters?page=11
Found 59 links

In [10]:
from tqdm import tqdm

In [13]:
article_links[-5:]

['https://reliefweb.int/disaster/ep-2013-000096-uga',
 'https://reliefweb.int/disaster/st-2009-000092-ind',
 'https://reliefweb.int/disaster/ep-2017-000187-bgd',
 'https://reliefweb.int/disaster/tc-1996-000008-phl',
 'https://reliefweb.int/disaster/fl-2003-000350-mng']

In [16]:
# "https://reliefweb.int/disaster/ep-2025-000017-uga" in article_links
article_links.index("https://reliefweb.int/disaster/ep-2025-000017-uga")

2219

In [20]:
# Phase 2: Visit each article and extract detailed information.
disaster_details = []

In [21]:
for link in tqdm(article_links):
    detail = scrape_disaster_detail(link)
    if detail:
        disaster_details.append(detail)
    # time.sleep(1)  # Polite delay between requests

    # Save collected data to JSON.
    with open("disaster_details.json", "w", encoding="utf-8") as f:
        json.dump(disaster_details, f, ensure_ascii=False, indent=2)
print("Scraping complete. Data saved to disaster_details.json.")

  0%|                                                                                         | 0/3589 [00:00<?, ?it/s]

Scraping detail page: https://reliefweb.int/disaster/ep-2020-000120-caf


  0%|                                                                                 | 1/3589 [00:00<46:52,  1.28it/s]

Scraping detail page: https://reliefweb.int/disaster/wf-2009-000162-grc


  0%|                                                                                 | 2/3589 [00:01<40:15,  1.49it/s]

...

Scraping detail page: https://reliefweb.int/disaster/tc-1996-000008-phl


100%|█████████████████████████████████████████████████████████████████████████████▉| 3588/3589 [55:48<00:01,  1.02s/it]

Scraping detail page: https://reliefweb.int/disaster/fl-2003-000350-mng


100%|██████████████████████████████████████████████████████████████████████████████| 3589/3589 [55:49<00:00,  1.07it/s]

Scraping complete. Data saved to disaster_details.json.
